In [1]:
import os

import duckdb

from streetscapes import conf

/Users/clairedonnelly/Documents/Urban-M4/test/streetscapes/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Convert CSV files to parquet and merging them together

The CSV files of the original Global Streetscapes dataset add up to 64GB in total. Moreover, data is split in several files which can make it a bit cumbersome to work with. Here, we convert the data to Parquet, which reduces file size and makes it easier to load and manipulate the data. 

Additionally, we combine columns from several sources into a single dataset that should serve most usecases.

In [2]:
# Convert all csvs in data dir to parquet 
for file in (conf.DATA_DIR / "data").glob("*.csv"):
    print(file)
    duckdb.sql(f"""
        COPY '{file}'
        TO '{file.with_suffix(".parquet")}' 
        (FORMAT 'parquet', COMPRESSION 'zstd')
    """)

/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/simplemaps.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/perception.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/osm.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/places365.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/segmentation.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/contextual.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/metadata_common_attributes.csv
/Users/clairedonnelly/Documents/Urban-M4/streetscapes-data/data/ghsl.csv


In [3]:
csv_size = sum(file.stat().st_size for file in (conf.DATA_DIR / "data").glob("*.csv") if file.is_file())
parquet_size = sum(file.stat().st_size for file in (conf.DATA_DIR / "data").glob("*.parquet") if file.is_file() and not file.name=="combined.parquet")

def human_readable(size):
    """Format byte size in human readable format"""
    order_of_magnitude = size.bit_length() // 10  # Dividing by 10 for base-1024 magnitude
    match order_of_magnitude:
        case 3:
            return f"{size / 1024**3:.2f} GB"
        case 2:
            return f"{size / 1024**2:.2f} MB"
        case 1:
            return f"{size / 1024:.2f} KB"
        case _:
            return f"{size} bytes"

print(f"{human_readable(csv_size)=}, {human_readable(parquet_size)=}")

reduction_factor = csv_size/parquet_size
print(f"{reduction_factor=}")

human_readable(csv_size)='13.09 GB', human_readable(parquet_size)='3.39 GB'
reduction_factor=3.8614643432586115


For some usecases it might be more convenient to combine several (if not all) columns in a single table. If we use JOIN like above on the full table, we quickly run into memory issues. This is because `duckdb.sql(...)` creates an in-memory database to load the data and keep track of intermediate results. Alternatively, duckdb can create a persistent database on disk using `duckdb.connect('database_filename')`. Here, we combine all csvs into a single parquet file. 

In [ ]:
files = [
        "contextual",
        "metadata_common_attributes",
        "segmentation",
        "simplemaps",
        "ghsl",
        "perception",
        "places365",
        "osm",
    ]

with duckdb.connect("duck.db", config={'threads': 1}) as con:
    for filename in files:
        con.sql(f"CREATE TABLE {filename} AS SELECT * FROM '{conf.DATA_DIR}/data/{filename}.parquet'")
    
    # Now we can perform the joins. We will do it step by step to keep checks on the memory needed to perform each join.
    for i, filename in enumerate(files[:-1]):
        # First we need to copy the content of each csv file into the database
        j = i + 1  
        con.sql(f"CREATE TABLE step{j} AS SELECT * FROM {filename} JOIN {files[j]} USING (UUID, source, orig_id)")     
        print(filename)
    
    # Finally, we can export the joined table to a new parquet file
    con.sql(f"COPY step7 TO '{conf.DATA_DIR}/data/combined.parquet' (FORMAT 'parquet', COMPRESSION 'zstd')")

# Remove the database from our filesystem
os.remove("duck.db")

# Show the combined file size:
combined_size = (conf.DATA_DIR / "data" / "combined.parquet").stat().st_size
print(f"{human_readable(combined_size)=}")


contextual
metadata_common_attributes
segmentation
simplemaps
ghsl
perception
places365
human_readable(combined_size)='0.53 GB'


In [ ]:
# Let's inspect the new file to see if the join has worked
duckdb.sql(f"SELECT * FROM '{conf.DATA_DIR}/data/combined.parquet'")

In [ ]:
# TODO:
# - Only keep columns that we think are relevant
# - Could join on UUID only by either dropping source and orig_id of all but the first dataset, or by only selecting columns of interest for each dataset
# - Could delete intermediate tables (e.g. step1 could be removed once step2 has been created)
#
# # Perhaps something like 
# selection = {
#     "contextual": ['source', 'orig_id'],
#     "osm.csv": ['road_width', 'type_highway'],
#     "simplemaps.csv": ['city'],
#     "metadata_common_attributes.csv": ['lat', 'lon']
# }
# for file, columns in selection:
#     con.sql(f"CREATE TABLE contextual AS SELECT {columns} FROM '{conf.DATA_DIR}/data/{file}.parquet'")

# for i, file in enumerate(selection.keys):
#     con.sql("CREATE TABLE step{i} AS SELECT * FROM contextual JOIN metadata USING (UUID, source, orig_id)")

# con.sql(f"COPY step{i} TO '{conf.DATA_DIR}/data/combined.parquet' (FORMAT 'parquet', COMPRESSION 'zstd')")

We are in touch with the developers of the original Open Streetscapes dataset to add these parquet files to the dataset on huggingface.